In [1]:
import os
import hashlib
from ice_encryption import convertFiles

In [2]:
def fetchScripts():
    """Load scripts to hash from the game code."""
    with open('../../game/server/bb2/GameChecksumManager.cpp', 'r', encoding='ansi') as f:
        scripts = []
        for line in f:
            line = line.replace('\t', '').replace('\r', '').replace('\n', '').replace(',', '').replace('\"', '').strip()
            if line.endswith('.txt'):
                scripts.append(line)
        return scripts
    return []

def get_digest(file_path):
    h = hashlib.sha1()
    with open(file_path, 'rb') as file:
        while True:
            chunk = file.read(h.block_size)
            if not chunk:
                break
            h.update(chunk)
    return h.hexdigest().upper()

In [3]:
STW = lambda x: r'D:\Dropbox\STEAMWORKS\content\brainbread2\brainbread2_shared\brainbread2\{}'.format(x).replace('\\', '/')

In [ ]:
template = '''"Checksums"
{
    "Scripts"
    {
%s1
    }
    "Maps"
    {
%s2
    }
    "Tags"
    {
        "developer tags" "http://infestus-game.com/hl2world/bb2data/developers.txt"
        "donator tags" "http://infestus-game.com/hl2world/bb2data/donators.txt"
        "tester tags" "http://infestus-game.com/hl2world/bb2data/testers.txt"
        "ban data" "http://infestus-game.com/hl2world/bb2data/bans.txt"
        "blacklist data" "http://infestus-game.com/hl2world/bb2data/blacklistedservers.txt"
    }
    "Events"
    {
        "XP"
        {
            "url" "http://worldtimeapi.org/api/ip"
            "value" "2.5"
        }
    }
}'''
print(template)

In [5]:
data = {}
excluded = ['example_map', 'tutorial']

for path, _, files in os.walk(STW("maps"), topdown=True):
    for f in [x.lower() for x in files if x.lower().endswith('.bsp') and x.lower()[:-4] not in excluded]:
        data[f[:-4]] = os.path.getsize('{}/{}'.format(path, f))

template = template.replace('%s2', '\n'.join(['\t\t\"{}\" \"{}\"'.format(k, v) for k, v in sorted(data.items())]))

In [6]:
checksummed_files = []
for f in fetchScripts():
    try:
        dig = get_digest(STW(f))
        checksummed_files.append('\t\t\"{}\" \"{}\"'.format(f, dig))
    except Exception as e:
        continue

template = template.replace('%s1', '\n'.join(checksummed_files))

In [7]:
with open('./checksums.txt', 'w') as f:
    f.write(template)

In [ ]:
print(template)

In [9]:
convertFiles('./', './', '.txt', '.bbd', 'F3QxBzK6')